In [42]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
import io
import random

!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [43]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [44]:
train = pd.read_parquet("../Dataset/train.parquet")
valid = pd.read_parquet("../Dataset/valid.parquet")

In [45]:
train.Target.values

array(['[BOTH] bernie sanders [SEP] elizabeth warren [EOTH]',
       '[BVIL] uk government [EVIL]',
       '[BH] thais [EH]  [BOTH] hong kong [EOTH]', ...,
       '[BOTH] msnbc [SEP] bernie sanders [SEP] democratic party [SEP] joe biden [SEP] democratic debate [EOTH]',
       '[BOTH] barack obama [EOTH]',
       '[BOTH] biden obama meme [SEP] john robinson [SEP] memes [SEP] joe biden [SEP] barack obama [EOTH]'],
      dtype=object)

In [46]:
from transformers import AutoTokenizer

special_tokens = ["<pad>","<bos>","<eos>"]
special_target_tokens = special_tokens + ["[EH]","[BH]","[EVIC]","[EVIC]","[BVIL]","[EVIL]","[BOTH]","[EOTH]"]

def get_training_corpus(col):
    dataset = train.copy()
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples[col]


old_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

src_tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus("OCR"), 52000, new_special_tokens =special_tokens)

trgt_tokenizer = old_tokenizer.train_new_from_iterator(get_training_corpus("Target"), 52000, new_special_tokens = special_target_tokens)
#tokenizer.encode("How is life braack obama", return_tensors = "pt", add_special_tokens = False)

In [47]:
def verify_tokens(_tokenizer):
    tensor = _tokenizer.encode("<bos> [EH] How is life barack obama abcdde [BH]", return_tensors = "pt", add_special_tokens = False)
    for _tensor in tensor.view(-1):
        print(_tensor, _tokenizer.decode(_tensor))
    
verify_tokens(trgt_tokenizer)

tensor(6) <bos>
tensor(8) [EH]
tensor(3058) how
tensor(442) is
tensor(1389) life
tensor(161) barack
tensor(156) obama
tensor(1307) abc
tensor(3272) ##dd
tensor(73) ##e
tensor(9) [BH]


In [48]:
len(src_tokenizer.vocab), len(trgt_tokenizer.vocab)

(25956, 7273)

In [49]:
src_vocab = src_tokenizer.vocab
trgt_vocab = trgt_tokenizer.vocab

In [50]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64
s_PAD_IDX =src_vocab['<pad>']
s_BOS_IDX = src_vocab['<bos>']
s_EOS_IDX = src_vocab['<eos>']

t_PAD_IDX =trgt_vocab['<pad>']
t_BOS_IDX = trgt_vocab['<bos>']
t_EOS_IDX = trgt_vocab['<eos>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
  src_batch, trgt_batch = [], []
  for (src_item, trgt_item) in data_batch:
    src_batch.append(torch.cat([torch.tensor([s_BOS_IDX])
        , src_tokenizer.encode(src_item, return_tensors = "pt", add_special_tokens = False).view(-1)
        , torch.tensor([s_EOS_IDX])], dim=0).type(torch.LongTensor))
    
    trgt_batch.append(torch.cat([torch.tensor([t_BOS_IDX])
        ,trgt_tokenizer.encode(trgt_item, return_tensors = "pt", add_special_tokens = False).view(-1) 
        , torch.tensor([t_EOS_IDX])], dim=0).type(torch.LongTensor))
  src_batch = pad_sequence(src_batch, padding_value=s_PAD_IDX)
  trgt_batch = pad_sequence(trgt_batch, padding_value=t_PAD_IDX)
  return src_batch, trgt_batch

train_data = list(zip(train.OCR.values, train.Target.values))
valid_data = list(zip(valid.OCR.values, valid.Target.values))

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(valid_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
#test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,shuffle=True, collate_fn=generate_batch)

In [51]:
next(iter(train_iter))

(tensor([[   6,    6,    6,  ...,    6,    6,    6],
         [ 337,  683, 3400,  ..., 1413,  795,  397],
         [6046, 1766,  991,  ...,  307, 2346, 3719],
         ...,
         [   5,    5,    5,  ...,    5,    5,    5],
         [   5,    5,    5,  ...,    5,    5,    5],
         [   5,    5,    5,  ...,    5,    5,    5]]),
 tensor([[   6,    6,    6,  ...,    6,    6,    6],
         [  11,   13,    7,  ...,   13,   13,   13],
         [ 161, 2921,    5,  ...,  128,  257, 1579],
         ...,
         [   5,    5,    5,  ...,    5,    5,    5],
         [   5,    5,    5,  ...,    5,    5,    5],
         [   5,    5,    5,  ...,    5,    5,    5]]))

In [52]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, dropout = dropout, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim, dropout=dropout)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))
        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

    def predict(self, src, start_token, end_token, max_len): 
        batch_size = src.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        #outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
        
        out = []
        current_token = start_token
        i = 0
        while (current_token.item() != end_token.item()) and (i<max_len):
            output, hidden = self.decoder(current_token, hidden, encoder_outputs)
            top1 = output.argmax(1) 
            current_token = top1
            out.append(top1.item())
            i = i+1
        return out


INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trgt_vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 64
DEC_EMB_DIM = 64
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 32
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.xavier_uniform_(param.data, gain=nn.init.calculate_gain('relu'))
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

optimizer = optim.Adam(model.parameters(), lr=0.0001)


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,109,705 trainable parameters


C:\Miniconda\envs\nlp\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [53]:
print(INPUT_DIM, OUTPUT_DIM)

25956 7273


In [54]:
#Note: when scoring the performance of a language translation model in particular, 
# we have to tell the nn.CrossEntropyLoss function to ignore the indices where the target is simply padding.

criterion = nn.CrossEntropyLoss(ignore_index=t_PAD_IDX)

In [55]:
import math
import time


def trainmodel(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = trainmodel(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

#test_loss = evaluate(model, test_iter, criterion)

#print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 0m 14s
	Train Loss: 8.754 | Train PPL: 6333.963
	 Val. Loss: 8.268 |  Val. PPL: 3897.322
Epoch: 02 | Time: 0m 14s
	Train Loss: 6.142 | Train PPL: 464.975
	 Val. Loss: 5.231 |  Val. PPL: 187.053
Epoch: 03 | Time: 0m 14s
	Train Loss: 4.670 | Train PPL: 106.651
	 Val. Loss: 5.242 |  Val. PPL: 189.034
Epoch: 04 | Time: 0m 14s
	Train Loss: 4.611 | Train PPL: 100.594
	 Val. Loss: 5.308 |  Val. PPL: 201.925
Epoch: 05 | Time: 0m 14s
	Train Loss: 4.570 | Train PPL:  96.565
	 Val. Loss: 5.258 |  Val. PPL: 192.057
Epoch: 06 | Time: 0m 14s
	Train Loss: 4.515 | Train PPL:  91.389
	 Val. Loss: 5.252 |  Val. PPL: 190.931
Epoch: 07 | Time: 0m 14s
	Train Loss: 4.457 | Train PPL:  86.250
	 Val. Loss: 5.246 |  Val. PPL: 189.854
Epoch: 08 | Time: 0m 14s
	Train Loss: 4.409 | Train PPL:  82.159
	 Val. Loss: 5.223 |  Val. PPL: 185.414
Epoch: 09 | Time: 0m 14s
	Train Loss: 4.366 | Train PPL:  78.694
	 Val. Loss: 5.156 |  Val. PPL: 173.544
Epoch: 10 | Time: 0m 14s
	Train Loss: 4.327 | Train P

In [56]:
torch.save(model.state_dict(), "../Models/model.pt")

In [57]:
checkpoint = torch.load("../Models/model.pt") 

In [58]:
model.load_state_dict(checkpoint)

<All keys matched successfully>

#### Inferencing

In [59]:
def translate(model, input_sentence, max_len):
    start_token = src_vocab['<bos>']
    end_token = src_vocab['<eos>']
    start_token = torch.LongTensor([start_token]).to(device)
    end_token = torch.LongTensor([end_token]).to(device)
    
    sent_tokens = torch.LongTensor( src_tokenizer.encode(input_sentence, return_tensors = "pt", add_special_tokens = False).view(-1, 1)).to(device)
    
    #Set the model to evaluation mode
    model.eval()
    with torch.no_grad():
        out = model.predict(sent_tokens, start_token, end_token, max_len)
        return out

In [60]:
input_sentence = valid.loc[5]["OCR"]

output_sentence = valid.loc[5]["Target"]
print(input_sentence)
print(output_sentence)

max_len = 35
out = translate(model, input_sentence, max_len)
print("Output Tokens : {}".format(out))
generated_text = trgt_tokenizer.decode(out)
print("Generated : {}".format(generated_text))

Dr Grayfang Dororayfang dpraytang - Foloning oytang "the best thing quarantine ice cream" ge veve seen theeeblble Yomno Soe e coure e conmolemoe por Ld aean 
[BOTH] dr grayfang [SEP] quarantine [SEP] ice cream [EOTH]
Output Tokens : [13, 128, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7]
Generated : [BOTH] donald [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] <eos>
